<a href="https://colab.research.google.com/github/wuliopulio/EmotionClassificationModel/blob/main/Facial_Expression_AffectNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer Learning with AffectNet
https://www.kaggle.com/datasets/mstjebashazida/affectnet/data

### Imports

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import os
from PIL import Image

In [ ]:
dtype = torch.float
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


### 1. Load and Pre-process Data

In [ ]:
import kagglehub

# Download latest version
image_data_path = kagglehub.dataset_download("mstjebashazida/affectnet")

print("Path to dataset files:", image_data_path)

Path to dataset files: /kaggle/input/affectnet


In [ ]:
class AFFECTNETDatasetTrain(Dataset):
  def __init__(self, data_dir, transform):
    super(AFFECTNETDatasetTrain).__init__()
    self.labels = os.listdir(data_dir)
    self.transform = transform

    classes = sorted(os.listdir(image_data_path))

    class_to_index = {}

    self.samples = []

    class_to_index = {
        "anger" : 2,
        "contempt" : 2,
        "disgust" : 5,
        "fear" : 1,
        "happy" : 6,
        "neutral" : 3,
        "sad" : 4,
        "surprise" : 0
    }

    for label in self.labels:
       for image_path in os.listdir(os.path.join(data_dir, label)):
          fpath = os.path.join(data_dir, label, image_path)
          label_index = class_to_index[label]
          self.samples.append((fpath, label_index))

  def __len__(self):
      return len(self.samples)

  def __getitem__(self, idx):
      image_path, label = self.samples[idx]
      image = Image.open(image_path)
      if self.transform is not None:
        image = self.transform(image)
      return image, label

In [ ]:
class AFFECTNETDatasetTest(Dataset):
  def __init__(self, data_dir, transform):
    super(AFFECTNETDatasetTest).__init__()
    self.labels = os.listdir(data_dir)
    self.transform = transform

    classes = sorted(os.listdir(image_data_path))

    class_to_index = {}

    self.samples = []

    class_to_index = {
        "Anger" : 2,
        "Contempt" : 2,
        "disgust" : 5,
        "fear" : 1,
        "happy" : 6,
        "neutral" : 3,
        "sad" : 4,
        "surprise" : 0
    }

    for label in self.labels:
       for image_path in os.listdir(os.path.join(data_dir, label)):
          fpath = os.path.join(data_dir, label, image_path)
          label_index = class_to_index[label]
          self.samples.append((fpath, label_index))

  def __len__(self):
      return len(self.samples)

  def __getitem__(self, idx):
      image_path, label = self.samples[idx]
      image = Image.open(image_path)
      if self.transform is not None:
        image = self.transform(image)
      return image, label

In [ ]:
import os
import torch
from torchvision import datasets
from torch.utils.data import random_split

train_transform = transforms.Compose([
    transforms.Resize((48, 48)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(45),
    transforms.RandomHorizontalFlip(),
    transforms.Grayscale(),
	  transforms.ToTensor()
])

test_transform = transforms.Compose([
     transforms.Resize((48, 48)),
    transforms.Grayscale(),
    transforms.ToTensor()
])

trainset = AFFECTNETDatasetTrain(os.path.join(image_data_path, 'archive (3)', 'Train'), train_transform)
testset = AFFECTNETDatasetTest(os.path.join(image_data_path, 'archive (3)', 'Test'), test_transform)

# Loaders
trainloader = DataLoader(trainset, batch_size=32, shuffle=True)
testloader = DataLoader(testset, batch_size=32, shuffle=False)

In [ ]:
class EmotionCNN(nn.Module):
  def __init__(self):
    super(EmotionCNN, self).__init__()

    # Define layers

    # Block 1:  48*48*3 -> 48 * 48 * 16 -> 24*24*16
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1)
    self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

    # Block 2: 24 * 24 * 16 -> 24 * 24 * 32 -> 12 * 12 * 32
    self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
    self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

    # Block 3: 12 * 12 * 32 -> 12 * 12 * 64 -> 6 * 6 * 64
    self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
    self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

    # Block 4: 6 * 6 * 64 -> 6 * 6 * 128 -> 3 * 3 * 128
    self.conv4 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
    self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)

    # FC layers for classification
    self.fc1 = nn.Linear(128*3*3, 512)
    self.dropout = nn.Dropout(p=0.5)
    self.fc2 = nn.Linear(512, 256)
    self.fc3 = nn.Linear(256, 7)

  def forward(self, x):
    # CONV -> RELU -> POOL

    # Block 1
    x = self.pool1(F.relu(self.conv1(x)))
    # Block 2
    x = self.pool2(F.relu(self.conv2(x)))
    # Block 3
    x = self.pool3(F.relu(self.conv3(x)))
    # Block 4
    x = self.pool4(F.relu(self.conv4(x)))

    # Flatten x
    x = x.view(x.size(0), -1)

    # MLP
    x = F.relu(self.fc1(x))
    x = self.dropout(x)
    x = F.relu(self.fc2(x))
    x = self.fc3(x)

    return x

In [ ]:
import torchvision.models as models

# Mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Correct full path
model_path = '/content/drive/MyDrive/Emotion Classifier CNN/Models/emotion_facial_model_1.pth'

# Load weights
model = EmotionCNN().to(device)
model.load_state_dict(torch.load(model_path, map_location=device))

<All keys matched successfully>

In [ ]:
def train_model(model, train_loader, criterion, optimizer, num_epochs, model_name='Model'):
  train_losses = []

  model.train()

  print(f"---Training {model_name} for {num_epochs} epochs---")

  for epoch in range(num_epochs):
    running_loss = 0.0
    correct = 0
    total = 0

    pbar = tqdm(train_loader, desc = f'Epoch {epoch + 1}/{num_epochs}') # displays a progress bar for the training
    for data, target in pbar:
      data, target = data.to(device), target.to(device) # moves both to the same device

      #Step 1. zero the gradient
      optimizer.zero_grad()

      #Step 2. Forward model
      output = model(data)

      # Step 3. Calculate the loss
      loss = criterion(output, target)

      # Step 4: Backward Pass
      loss.backward()

      # Step 5: Update the weights
      optimizer.step()

      running_loss += loss.item()
      _, predicted = torch.max(output.data, 1) # Gets the predicted class with the highest probability for each example.

      #Updates the total number of samples and correct predictions.
      total += target.size(0)
      correct += (predicted == target).sum().item()

      pbar.set_postfix({'Loss': f'{running_loss/(pbar.n+1):.4f}', 'Acc': f'{100.*correct/total:.2f}%'})

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100. * correct/total #Computes the average loss and accuracy for the full epoch.
    train_losses.append(epoch_loss)

  print(f'--- Finished training {model_name} ---')
  return train_losses

In [ ]:
def evaluate_model(model,test_loader):
  model.eval()
  correct = 0
  total = 0

  with torch.no_grad():
    for data, target in test_loader:
      data, target = data.to(device), target.to(device)
      output = model(data)
      _, predicted = torch.max(output.data, 1 )
      total += target.size(0)
      correct += (predicted == target).sum().item()

  return 100. * correct/total

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer_cnn = optim.Adam(model.parameters(), lr = 0.001)

cnn_losses = train_model(model, trainloader, criterion, optimizer_cnn, num_epochs= 30, model_name = 'Emotions CNN')
cnn_accuracy = evaluate_model(model, testloader)
print(f'Accuracy of Emotions CNN:{cnn_accuracy:.2f}%')

---Training Emotions CNN for 30 epochs---


Epoch 30/30: 100%|██████████| 504/504 [00:24<00:00, 20.80it/s, Loss=1.0406, Acc=58.87%]


--- Finished training Emotions CNN ---
Accuracy of Emotions CNN:57.29%


In [ ]:
# Get numbers of parameters
cnn_params = sum(p.numel() for p in model.parameters())

print(f'Test accuracy (CNN): {cnn_accuracy:.2f}%')

print(f'Number of parameters (CNN): {cnn_params}')

Test accuracy (CNN): 57.29%
Number of parameters (CNN): 820615


### 4. Save Model

In [ ]:
save_to = os.path.join('drive', 'MyDrive', 'Emotion Classifier CNN', 'Models', "emotion_facial_model_2.pth")
os.makedirs(os.path.dirname(save_to), exist_ok=True)
torch.save(model.state_dict(), save_to)